### Expense Tracker Application

In [22]:
import json
import pandas as pd
from tkinter import Tk, Label, Entry, Button, StringVar, messagebox, OptionMenu, Listbox, Scrollbar, END
import os
import datetime

# Class to manage expense tracking
class ExpenseTracker:
    def __init__(self):
        self.expenses = []  # Initialize with an empty list to start fresh
        self.load_data()

    def add_expense(self, amount, description, category):
        """Add an expense to the tracker."""
        expense = {
            'amount': amount,
            'description': description,
            'category': category,
            'date': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        self.expenses.append(expense)
        self.save_data()
        messagebox.showinfo("Success", "Expense added successfully!")

    def delete_expense(self, index):
        """Delete an expense from the tracker."""
        try:
            self.expenses.pop(index)
            self.save_data()
            messagebox.showinfo("Success", "Expense deleted successfully!")
        except IndexError:
            messagebox.showerror("Error", "Invalid selection. Please select a valid expense to delete.")

    def save_data(self):
        """Save expenses data to a JSON file."""
        with open('expenses.json', 'w') as file:
            json.dump(self.expenses, file, indent=4)
        print("Data saved successfully!")

    def load_data(self):
        """Ensure no data is present initially."""
        self.expenses = []  # Start with an empty list
        if os.path.exists('expenses.json'):
            os.remove('expenses.json')  # Remove the existing file to start fresh
            print("Previous data file found and removed, starting fresh.")
        else:
            print("No previous data file found, starting fresh.")

    def view_monthly_summary(self, month):
        """Display a monthly summary of expenses in a message box."""
        df = pd.DataFrame(self.expenses)
        if df.empty:
            messagebox.showinfo("No Data", "No expenses to show.")
            return
        
        df['date'] = pd.to_datetime(df['date'])
        monthly_expenses = df[df['date'].dt.to_period('M') == month]
        total = monthly_expenses['amount'].sum()

        if not monthly_expenses.empty:
            summary = f"Monthly Summary for {month}:\n\n"
            for idx, row in monthly_expenses.iterrows():
                summary += f"{row['date'].strftime('%Y-%m-%d %H:%M:%S')} - {row['amount']} - {row['description']} - {row['category']}\n"
            summary += f"\nTotal Expense: {total}"
            messagebox.showinfo("Monthly Summary", summary)
        else:
            messagebox.showinfo("No Data", f"There are no expenses recorded for the month: {month}.")

    def view_category_summary(self):
        """Display a summary of expenses by category in a message box."""
        df = pd.DataFrame(self.expenses)
        if df.empty:
            messagebox.showinfo("No Data", "No expenses to show.")
            return
        
        category_totals = df.groupby('category')['amount'].sum().reset_index()
        summary = "Expenses by Category:\n\n"
        for idx, row in category_totals.iterrows():
            summary += f"{row['category']}: {row['amount']}\n"
        
        messagebox.showinfo("Category Summary", summary)

# Function to add an expense using GUI input
def add_expense():
    try:
        amount = float(amount_var.get())
        if amount <= 0:
            raise ValueError("Amount must be greater than zero.")
        description = description_var.get().strip()
        category = category_var.get().strip()
        if not description or not category:
            raise ValueError("Description and Category cannot be empty.")
        tracker.add_expense(amount, description, category)
        amount_var.set("")
        description_var.set("")
        refresh_expense_list()
    except ValueError as e:
        messagebox.showerror("Invalid Input", str(e))

# Function to delete selected expense from listbox
def delete_expense():
    selected_idx = expense_listbox.curselection()
    if selected_idx:
        tracker.delete_expense(selected_idx[0])
        refresh_expense_list()
    else:
        messagebox.showerror("Error", "Please select an expense to delete.")

# Function to refresh the listbox with current expenses
def refresh_expense_list():
    expense_listbox.delete(0, END)
    for expense in tracker.expenses:
        expense_listbox.insert(END, f"{expense['date']} - {expense['amount']} - {expense['description']} - {expense['category']}")

# Setting up the GUI
tracker = ExpenseTracker()
root = Tk()
root.title("Expense Tracker")

amount_var = StringVar()
description_var = StringVar()
category_var = StringVar()

Label(root, text="Amount:").grid(row=0, column=0, padx=10, pady=10)
Entry(root, textvariable=amount_var).grid(row=0, column=1, padx=10, pady=10)

Label(root, text="Description:").grid(row=1, column=0, padx=10, pady=10)
Entry(root, textvariable=description_var).grid(row=1, column=1, padx=10, pady=10)

Label(root, text="Category:").grid(row=2, column=0, padx=10, pady=10)
categories = ["Food", "Transportation", "Entertainment", "Utilities", "Others"]
category_var.set(categories[0])
OptionMenu(root, category_var, *categories).grid(row=2, column=1, padx=10, pady=10)

Button(root, text="Add Expense", command=add_expense).grid(row=3, column=0, columnspan=2, pady=10)
Button(root, text="Delete Selected Expense", command=delete_expense).grid(row=4, column=0, columnspan=2, pady=10)

# Entry for month selection
month_var = StringVar()
month_var.set("2024-08")  # Default value
Label(root, text="Select Month (YYYY-MM):").grid(row=5, column=0, padx=10, pady=10)
Entry(root, textvariable=month_var).grid(row=5, column=1, padx=10, pady=10)

Button(root, text="View Monthly Summary", command=lambda: tracker.view_monthly_summary(month_var.get())).grid(row=6, column=0, columnspan=2, pady=10)
Button(root, text="View Category Summary", command=tracker.view_category_summary).grid(row=7, column=0, columnspan=2, pady=10)

expense_listbox = Listbox(root, width=50, height=10)
expense_listbox.grid(row=8, column=0, columnspan=2, pady=10)

scrollbar = Scrollbar(root)
scrollbar.grid(row=8, column=2, sticky='ns')
expense_listbox.config(yscrollcommand=scrollbar.set)
scrollbar.config(command=expense_listbox.yview)

refresh_expense_list()
root.mainloop()


No previous data file found, starting fresh.
